In [1]:
!pip install lmfit

  Created wheel for lmfit: filename=lmfit-1.0.1-py3-none-any.whl size=82006 sha256=104e88d9223f17e65ad56481d10f965fcca29694962bf1b6c77a06bdf845bf9f
  Stored in directory: c:\users\fmill\appdata\local\pip\cache\wheels\b6\e0\3d\52a095d9f869ec51e0f8052e4e80bf8e8d844d574e3f7d214b
  Created wheel for asteval: filename=asteval-0.9.21-py3-none-any.whl size=17299 sha256=32d25b7cc16a76536c1af757fa744d83867d45ba824eaeae4c271e60bb1b9f2b
  Stored in directory: c:\users\fmill\appdata\local\pip\cache\wheels\83\4d\3c\06c0b065829c90952351897b82bc35eef1fe5155cdd89dc032
Successfully built lmfit asteval


In [86]:
import sklearn
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model


import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv',
                error_bad_lines=False
                 )

df= df.set_index('Fecha')

In [6]:
df.head()

,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales,Casos activos por FD,Casos activos por FIS,Casos recuperados por FIS,Casos recuperados por FD,Casos confirmados recuperados,Casos activos confirmados,Casos probables acumulados,Casos activos probables,Casos nuevos sin notificar,Casos \nactivos confirmados
Fecha,,,,,,,,,,,,,,,,,
2020-03-02,1.0,1.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-03,0.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-04,2.0,3.0,0.0,0.0,3.0,NaN,2.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-05,1.0,4.0,0.0,0.0,4.0,NaN,1.0,4.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-06,1.0,5.0,0.0,0.0,5.0,NaN,1.0,5.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
real_data = np.array(df['Casos totales'].rolling(14).mean().fillna(0))
y_data = np.concatenate((np.zeros(30), real_data))
days = 30 + len(real_data)
x_data = np.linspace(0, days - 1, days, dtype=int) 

In [99]:
X_train, X_test, y_train, y_test = train_test_split(y_data, x_data, test_size=0.25, random_state=42, shuffle=False)

In [100]:
u, c = np.unique(y_train, return_counts=True)
print(u,'\n',c)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [101]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [105]:

parameters = {'n_samples':[3]}

In [106]:
model = linear_model.Ridge(alpha=.5)
scoring = ["f1_macro", "recall_macro"]
grid_search = GridSearchCV(model, parameters, cv=3, scoring=scoring, refit=False)

In [107]:
linear_model.Ridge(alpha=.5).get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'random_state', 'solver', 'tol'])

In [108]:
%%time
grid_search = grid_search.fit([X_train, y_train])

ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=2.

In [95]:
results_cv = pd.DataFrame(grid_search.cv_results_)
results_cv = results_cv[['params', 'mean_test_f1_macro','std_test_f1_macro']]
results_cv = results_cv.sort_values('mean_test_f1_macro', ascending=False)

In [96]:
results_cv

,params,mean_test_f1_macro,std_test_f1_macro
0,{},NaN,NaN


In [97]:
results_cv.iloc[0, :]['params']

{}

Adicionalmente, una vez hayamos definido los hiperparametros podemos hacer

In [76]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
import numpy as np

In [161]:
rf = RandomForestClassifier(n_estimators=200, criterion='entropy', class_weight='balanced')

In [164]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [169]:
results = cross_validate(rf, X_train, y_train, cv=cv, return_estimator=True, scoring='f1_macro')

In [170]:
results

{'fit_time': array([0.29935932, 0.28405952, 0.28638458, 0.32385039, 0.34898949]),
 'score_time': array([0.0205276 , 0.02039266, 0.01897836, 0.02665019, 0.02451134]),
 'estimator': (RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200)),
 'test_score': array([0.91534392, 1.        , 0.93939394, 0.93611111, 0.86425339])}

o tambien podriamos hace un ciclo for e iterar sobre los subset que genera `ShuffleSplit()`

In [174]:
val_f1s = results['test_score']

In [176]:
print('VAL SCORE: {:.2f} +- {:.2f}'.format(np.mean(val_f1s), np.std(val_f1s)))

VAL SCORE: 0.93 +- 0.04


## Final Model
El mejor modelo (osea el que ocuparemos para produccion) puede ser aquel que obtuvo el mejor f1 en el conjunto de test

In [177]:
f1_list = []
for model in results['estimator']:
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    f1_list.append(f1)

In [178]:
print('{:.2f} + {:.2f}'.format(np.mean(f1_list), np.std(f1_list)))

0.99 + 0.01


Esta metrica nos permite discriminar entre un modelo u otro

In [ ]:
# 